# 使用 AG News 数据集创建 Amazon Comprehend 自定义分类任务

## 安装并导入依赖包

In [ ]:
!pip install --upgrade  s3fs pandas  tqdm

In [ ]:
import pandas as pd
import tqdm
import boto3
region_name='us-east-1'
import matplotlib

### 获取训练数据，我们的数据位于 Amazon S3 中。
### 数据是一个 tar 格式的压缩包，下载该文件并进行解压。 

### 可以忽略 untar 操作中来自 perssions 的消息。

In [ ]:
! wget https://s3.amazonaws.com/fast-ai-nlp/ag_news_csv.tgz
! tar xvzf ag_news_csv.tgz

#### 使用Pandas读取csv文件

In [ ]:
train=pd.read_csv("ag_news_csv/train.csv", names=['category','title','text'])

#### 这是我们的训练数据集。 它有 3 列，标签、标题和文本。 

In [ ]:
train

#### 为了将训练时间减少，我们采样出 1000 行数据。

In [ ]:
train = train.sample(axis='index',n=1000,random_state=100)

#### 将category的数字转化为实际的类别名称

In [ ]:
labeldict={'1': 'WORLD', '2' :  'SPORTS', '3' : 'BUSINESS', '4': 'SCI_TECH'}
trainstr=train.astype(str)
trainstr['label']=trainstr['category'].replace(labeldict)

#### 将标题和文本放在一栏中，以便我们进行训练。

#### 现在，只使用标签和文本列，因为这是 Comprehend 期望的输入。 

In [ ]:
dfout=trainstr[["label", 'text']]  

In [ ]:
dfout

#### 让我们看下各个标签的统计。样本非常均衡。

In [ ]:
dfout['label'].value_counts()

### 将数据复制到 S3 存储桶 


In [ ]:
# Get the account ID from STS so we can all have unique bucket names
client = boto3.client("sts")
account_id = client.get_caller_identity()["Account"]
bucket_name = "comprehend-labs" + account_id +  "-2"
print ("Bucket name used is " + bucket_name )

In [ ]:
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')

if (s3.Bucket(bucket_name).creation_date is None):
    #location = {'LocationConstraint': region_name}
    s3_client.create_bucket(Bucket=bucket_name)#, CreateBucketConfiguration=location)
    print ("Created bucket " + bucket_name)
else:
    print ("Bucket Exists")

In [ ]:
file_name="s3://" + bucket_name + "/custom_news_classification.csv"

In [ ]:
dfout.to_csv(file_name, header=False, index=False )

### 返回 Workshop 手册继续实验，根据手册要求将以下内容复制到创建自定义分类器界面上S3 location的位置

In [ ]:
print(file_name)